EXTRACT

In [71]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd 
import numpy as np

In [72]:
load_dotenv()

data_path = os.getenv("RAW_DATA_PATH")
df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

TRANSFORM


In [73]:
#create tables

client_table = df[['client_id','age','job','marital','education','credit_default','mortgage']].copy()
campaign_table = df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome','month','day']].copy()
economics_table = df[['client_id','cons_price_idx','euribor_three_months']].copy()

In [74]:
#cleaning requirements
#Client_table

client_table['job'] = client_table['job'].str.replace('.','_')
client_table['education'] = client_table['education'].str.replace('.','_').replace('unknown',np.nan)
for col in ['credit_default','mortgage']:
    client_table[col] = client_table[col].map({ 'yes': 1,
                                               'no':0,
                                               'unknown':0})
    client_table[col] = client_table[col].astype(bool)


In [75]:
#Campaign_table

campaign_table['previous_outcome'] = campaign_table['previous_outcome'].map({'success':1,
                                                                             'failure':0,
                                                                             'nonexistent':0})
campaign_table['campaign_outcome'] = campaign_table['campaign_outcome'].map({'yes':1,
                                                                             'no':0})
campaign_table['previous_outcome'] = campaign_table['previous_outcome'].astype(bool)
campaign_table['campaign_outcome'] = campaign_table['campaign_outcome'].astype(bool)

campaign_table['year'] = "2022"
campaign_table['day'] = campaign_table['day'].astype(str)
campaign_table['last_contact_date'] = campaign_table['year'] + '-' + campaign_table['month'] + '-' + campaign_table['day']
campaign_table['last_contact_date'] = pd.to_datetime(campaign_table['last_contact_date'],format="%Y-%b-%d")
campaign_table.drop(columns = ['month','year','day'], inplace = True)
campaign_table.insert(0, 'campaign_id',range(1,len(campaign_table)+1))

In [76]:
economics_table.insert(0,'economics_id',range(1,len(economics_table)+1))

LOAD


In [82]:
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

client_table.to_sql('client', engine, if_exists="append",index=False)
campaign_table.to_sql('campaign', engine, if_exists="append",index=False)
economics_table.to_sql('economics', engine, if_exists="append",index=False)

188